## Yelp生成ground truth
Las Vegas城市category层次树，共有21个top category，分别为...

In [1]:
import warnings
import pandas as pd
import json
from tqdm import tqdm
import math

with open('./data/yelp_category.json', 'r', encoding='utf-8') as f:
    json_data = json.loads(f.read())
    
hierarchy_data = pd.DataFrame(json_data)
hierarchy_data = hierarchy_data[['alias', 'title', 'parents']]
hierarchy_data['depth'] = ''
for m in tqdm(range(len(hierarchy_data))):
    depth = 1
    parent = hierarchy_data.loc[m]['parents']
    while len(parent) != 0:
        parent_alias = parent[0]
        parent = list(hierarchy_data[hierarchy_data.alias == parent_alias]['parents'])
        if len(parent) != 0:  # 玄学            
            parent = parent[0]
        depth += 1
    hierarchy_data.loc[m]['depth'] = depth

hierarchy_data = hierarchy_data.sort_values(by='depth')  # 根据深度排序，便于构建树
hierarchy_data.reset_index(drop=True, inplace=True)

100%|████████████████████████████████████████████████████████████████████████████| 1565/1565 [00:00<00:00, 1566.66it/s]


In [2]:
hierarchy_data

,alias,title,parents,depth
0,massmedia,Mass Media,[],1
1,localflavor,Local Flavor,[],1
2,pets,Pets,[],1
3,professional,Professional Services,[],1
4,eventservices,Event Planning & Services,[],1
...,...,...,...,...
1560,karate,Karate,[martialarts],4
1561,willstrustsprobates,"Wills, Trusts, & Probates",[estateplanning],4
1562,brazilianjiujitsu,Brazilian Jiu-jitsu,[martialarts],4
1563,gyudon,Gyudon,[donburi],4


In [10]:
import treelib
from treelib import Tree, Node
tree_loc=Tree()      
tree_loc.create_node("Root","root")
for m in tqdm(range(len(hierarchy_data))):
    _id = hierarchy_data.loc[m]['alias']
    _name = hierarchy_data.loc[m]['title']
    if len(hierarchy_data.loc[m]['parents']) == 0:
        _parent = 'root'
    else:
        _parent = hierarchy_data.loc[m]['parents'][0]
    tree_loc.create_node(_name,_id,parent=_parent)

NameError: name 'hierarchy_data' is not defined

In [4]:
tree_loc.show()

Root
├── Active Life
│   ├── ATV Rentals/Tours
│   ├── Airsoft
│   ├── Amateur Sports Teams
│   ├── Amusement Parks
│   ├── Aquariums
│   ├── Archery
│   ├── Axe Throwing
│   ├── Badminton
│   ├── Baseball Fields
│   ├── Basketball Courts
│   ├── Bathing Area
│   ├── Batting Cages
│   ├── Beach Equipment Rentals
│   ├── Beach Volleyball
│   ├── Beaches
│   ├── Bicycle Paths
│   ├── Bike Parking
│   ├── Bike Rentals
│   ├── Boating
│   ├── Bobsledding
│   ├── Bocce Ball
│   ├── Bowling
│   ├── Bubble Soccer
│   ├── Bungee Jumping
│   ├── Canyoneering
│   ├── Carousels
│   ├── Challenge Courses
│   ├── Climbing
│   ├── Cycling Classes
│   ├── Dart Arenas
│   ├── Day Camps
│   ├── Disc Golf
│   ├── Diving
│   │   ├── Free Diving
│   │   └── Scuba Diving
│   ├── Escape Games
│   ├── Experiences
│   ├── Fencing Clubs
│   ├── Fishing
│   ├── Fitness & Instruction
│   │   ├── Aerial Fitness
│   │   ├── Barre Classes
│   │   ├── Boot Camps
│   │   ├── Boxing
│   │   ├── Cardio Classes
│   │   

In [11]:
def findparent(id1,id2,tree_loc): #找到这俩id的共同父节点 
    if id1==id2:
        return id1
    id1_parent=tree_loc.parent(id1).identifier
    id2_parent=tree_loc.parent(id2).identifier
    if id1==id2_parent:   #id相同返回本身、一个是另一个的父亲则返回该结点
        return id1
    if id2==id1_parent:
        return id2
    a=0
    for i in range(5):      #循环、找到公共父节点
        id2_parent=tree_loc.parent(id2).identifier
        for j in range(5):
            if id1_parent==id2_parent:
                return id2_parent
            if id2_parent=='root':
                break;
            id2_parent=tree_loc.parent(id2_parent).identifier
        if a!=0:
            break
        id1_parent=tree_loc.parent(id1_parent).identifier
    return id1_parent


def findpath(id1,id2,tree_loc): #找到这俩id的最短路径    方法为两节点的深度减去两倍父节点深度
    path=0
    if id1==id2:
        return 0
    if id1=="root":
        return tree_loc.depth(id2)
    if id2=="root":
        return tree_loc.depth(id1)
    parent=findparent(id1,id2,tree_loc)
    if parent==id1 or parent==id2:
        return 1
    else:#找到这俩数据的父节点信息
        parent_class=tree_loc.depth(parent)
        id1_class=tree_loc.depth(id1)
        id2_class=tree_loc.depth(id2)
        path=id1_class+id2_class-2*parent_class
    return path

In [7]:
def output_695_correct_responses(data, name):
    queries = list(data[0])
    correct_responses = list(data[1])
    for i in range(len(correct_responses)):
        correct_responses[i] = correct_responses[i].split('@')[0]
    qr_dict = dict(zip(queries, correct_responses))

    with open('./data/categories_used.txt', 'r', encoding='utf-8') as f:
        categories = f.readlines()
    for i in range(len(categories)):
        categories[i] = categories[i].strip()

    responses = []
    for category in categories:
        query = hierarchy_data[hierarchy_data.title == category]['alias'].iloc[0]
        response_alias = qr_dict[query]
        response = hierarchy_data[hierarchy_data.alias == response_alias]['title'].iloc[0]
        responses.append(response)

    with open('ground_truth/' + name + '.txt', 'a') as f:
        for r in responses:
            f.write(r + "\n")

In [8]:
def get_SIM_WP(tree, data):
    sim_wp=[]  #[[t1,[t2,value],[t3,value]...]...]
    for i in tqdm(range(len(data)), desc='SIM_WP'):
        sim_wp_col=[]
        for j in range(len(data)):
            if i==j:
                continue
            else:
                parent_id=findparent(data.loc[i,"alias"],data.loc[j,"alias"],tree)
                n1=findpath(data.loc[i,"alias"],parent_id,tree)
                n2=findpath(data.loc[j,"alias"],parent_id,tree)
                n3=tree_loc.depth(parent_id)
                sim=2*n3/(n1+n2+2*n3)
                n=[data.loc[j,"alias"],sim]
            sim_wp_col.append(n)
        sim_wp_col=sorted(sim_wp_col,key=(lambda x:x[1]),reverse=True)   # 对数值进行排序
        sim_wp_col1=[item[0]+"@"+str(item[1]) for item in sim_wp_col]     # 按格式存储
        sim_wp_col=[data.loc[i,"alias"]]+sim_wp_col1
        sim_wp.append(sim_wp_col)
    df = pd.DataFrame(sim_wp)
    return df

In [9]:
def get_SIM_LC(tree, data):
    sim_lc=[]  # [[t1,[t2,value],[t3,value]...]...]为数据格式
    for i in tqdm(range(len(data)), desc='SIM_LC'):
        sim_lc_col=[]
        for j in range(len(data)):
            if i==j:
                continue
            else:
                n=findpath(data.loc[i,"alias"],data.loc[j,"alias"],tree)
                sim=-math.log(n/10)
                n=[data.loc[j,"alias"],sim]
                sim_lc_col.append(n)
        sim_lc_col=sorted(sim_lc_col,key=(lambda x:x[1]),reverse=True)   # 对数值进行排序
        sim_lc_col1=[item[0]+"@"+str(item[1]) for item in sim_lc_col]
        sim_lc_col=[data.loc[i,"alias"]]+sim_lc_col1
        sim_lc.append(sim_lc_col)
    df = pd.DataFrame(sim_lc)
    return df

In [10]:
def get_IC_Sanchez(tree, data):
    ic_sanchez_dict_name=[]
    ic_sanchez_dict_values=[]
    for i in range(len(data)):
        id=data.loc[i,"alias"]
        ic_sanchez_dict_name=ic_sanchez_dict_name+[id]
        leaves=len(tree.leaves(id))   # id下所有叶子结点
        subsumers=0
        if id=="root":
            subsumers=1
        else:
            subsumers=tree.depth(id)   # 比id更概括的类型
        max_leaves=len(tree_loc.leaves('root'))     # 所有叶子数
        ic=-math.log((abs(leaves)/abs(subsumers)+1)/(max_leaves+1))
        ic_sanchez_dict_values=ic_sanchez_dict_values+[ic]
    ic_sanchez_dict=dict(zip(ic_sanchez_dict_name,ic_sanchez_dict_values))
    ic_sanchez_dict.update({'root':-math.log((abs(max_leaves)+1)/(max_leaves+1))})  # data文件中没有root，单独把root加入
    return ic_sanchez_dict

In [11]:
def get_IC_Seco(tree, data):
    ic_seco_dict_name=[]       # IC_seco 的字典！！！
    ic_seco_dict_values=[]
    for i in range(len(data)):
        id=data.loc[i,"alias"]
        ic_seco_dict_name=ic_seco_dict_name+[id]
        hypo=len(tree_loc.subtree(id))-1   # id下所有结点
        max_types=len(tree)    # 所有节点
        ic=1-math.log(abs(hypo)+1)/math.log(max_types)
        ic_seco_dict_values=ic_seco_dict_values+[ic]
    ic_seco_dict=dict(zip(ic_seco_dict_name,ic_seco_dict_values))
    ic_seco_dict.update({'root':1-math.log(abs(max_types))/math.log(max_types)})   # data文件中没有root，单独把root加入
    return ic_seco_dict

In [12]:
def get_SIM_Lin(tree, data, IC_dict):  
    sim_lin=[]
    for i in tqdm(range(len(data)), desc='SIM_Lin'):
        sim_lin_col=[]
        for j in range(len(data)):
            if i==j:
                continue
            id1=data.loc[i,"alias"]
            id2=data.loc[j,"alias"] 
            parent=findparent(id1,id2,tree)     # 两id的最小共同父节点
            ic_parent=IC_dict[parent]
            ic_t1=IC_dict[id1]
            ic_t2=IC_dict[id1]
            sim=2*ic_parent/(ic_t1+ic_t2)
            sim_lin_col.append([id2,sim])
        sim_lin_col=sorted(sim_lin_col,key=(lambda x:x[1]),reverse=True)
        sim_lin_col1=[item[0]+"@"+str(item[1]) for item in sim_lin_col]
        sim_lin_col=[data.loc[i,"alias"]]+sim_lin_col1
        sim_lin.append(sim_lin_col)
    df = pd.DataFrame(sim_lin)
    return df

In [13]:
def get_SIM_JC(tree, data, IC_dict):     
    sim_jc=[]
    for i in tqdm(range(len(data)), desc='SIM_JC'):
        sim_jc_col=[]
        for j in range(len(data)):
            if i==j:
                continue
            id1=data.loc[i,"alias"]
            id2=data.loc[j,"alias"]
            parent=findparent(id1,id2,tree)
            ic_parent=IC_dict[parent]
            ic_t1=IC_dict[id1]
            ic_t2=IC_dict[id2]
            sim=ic_t1+ic_t2-(2*ic_parent)
            sim=1/sim
            sim_jc_col.append([data.loc[j,"alias"],sim])
        sim_jc_col=sorted(sim_jc_col,key=(lambda x:x[1]),reverse=True)
        sim_jc_col1=[item[0]+"@"+str(item[1]) for item in sim_jc_col]
        sim_jc_col=[data.loc[i,"alias"]]+sim_jc_col1
        sim_jc.append(sim_jc_col)
    df = pd.DataFrame(sim_jc)
    return df

In [14]:
IC_Sanchez_dict = get_IC_Sanchez(tree_loc, hierarchy_data)
IC_Seco_dict = get_IC_Seco(tree_loc, hierarchy_data)

In [15]:
SIM_WP = get_SIM_WP(tree_loc, hierarchy_data)
SIM_LC = get_SIM_LC(tree_loc, hierarchy_data)
SIM_Lin_Sanchez = get_SIM_Lin(tree_loc, hierarchy_data, IC_Sanchez_dict)
SIM_Lin_Seco = get_SIM_Lin(tree_loc, hierarchy_data, IC_Seco_dict)
SIM_JC_Sanchez = get_SIM_JC(tree_loc, hierarchy_data, IC_Sanchez_dict)
SIM_JC_Seco = get_SIM_JC(tree_loc, hierarchy_data, IC_Seco_dict)


SIM_JC: 100%|██████████████████████████████████████████████████████████████████████| 1565/1565 [03:30<00:00,  7.43it/s]


In [20]:
output_695_correct_responses(SIM_WP, 'SIM_WP')
output_695_correct_responses(SIM_LC, 'SIM_LC')
output_695_correct_responses(SIM_Lin_Sanchez, 'SIM_Lin_Sanchez')
output_695_correct_responses(SIM_Lin_Seco, 'SIM_Lin_Seco')
output_695_correct_responses(SIM_JC_Sanchez, 'SIM_JC_Sanchez')
output_695_correct_responses(SIM_JC_Seco, 'SIM_JC_Seco')


## FourSquare生成ground truth
基于New York城市的category层次树，共有10个category,分别为

['Arts & Entertainment', 'College & University', 'Event', 'Food', 'Nightlife Spot', 'Outdoors & Recreation', 'Professional & Other Places', 'Residence', 'Shop & Service', 'Travel & Transport']

In [23]:
import treelib
from treelib import Tree, Node
import pandas as pd
from tqdm import tqdm
import math
tree_loc=Tree()          #将所有类别建立成一个树，在这里，我事先把所有的类别加上了深度并且排序完毕，方便树的建立
data_loc = pd.read_excel('./data/foursquare_category.xlsx') 
tree_loc.create_node("Root","root")
for m in range(0, len(data_loc)):
    a=",".join(str(i) for i in data_loc.loc[m])
    arr=a.split(",")
    _id=arr[0]
    _name=arr[1]
    _parent=(arr[2].split("$"))[0]
    tree_loc.create_node(_name,_id,parent=_parent)
tree_loc.show()

Root
├── Arts & Entertainment
│   ├── Amphitheater
│   ├── Aquarium
│   ├── Arcade
│   ├── Art Gallery
│   ├── Bowling Alley
│   ├── Casino
│   ├── Circus
│   ├── Comedy Club
│   ├── Concert Hall
│   ├── Country Dance Club
│   ├── Disc Golf
│   ├── Exhibit
│   ├── General Entertainment
│   ├── Go Kart Track
│   ├── Historic Site
│   ├── Karaoke Box
│   ├── Laser Tag
│   ├── Memorial Site
│   ├── Mini Golf
│   ├── Movie Theater
│   │   ├── Drive-in Theater
│   │   ├── Indie Movie Theater
│   │   └── Multiplex
│   ├── Museum
│   │   ├── Art Museum
│   │   ├── Erotic Museum
│   │   ├── History Museum
│   │   ├── Planetarium
│   │   └── Science Museum
│   ├── Music Venue
│   │   ├── Jazz Club
│   │   ├── Piano Bar
│   │   └── Rock Club
│   ├── Pachinko Parlor
│   ├── Performing Arts Venue
│   │   ├── Dance Studio
│   │   ├── Indie Theater
│   │   ├── Opera House
│   │   └── Theater
│   ├── Pool Hall
│   ├── Public Art
│   │   ├── Outdoor Sculpture
│   │   └── Street Art
│   ├── Racecourse


In [24]:
tree_loc.leaves()

[Node(tag=Amphitheater, identifier=56aa371be4b08b9a8d5734db, data=None),
 Node(tag=Aquarium, identifier=4fceea171983d5d06c3e9823, data=None),
 Node(tag=Arcade, identifier=4bf58dd8d48988d1e1931735, data=None),
 Node(tag=Art Gallery, identifier=4bf58dd8d48988d1e2931735, data=None),
 Node(tag=Bowling Alley, identifier=4bf58dd8d48988d1e4931735, data=None),
 Node(tag=Casino, identifier=4bf58dd8d48988d17c941735, data=None),
 Node(tag=Circus, identifier=52e81612bcbc57f1066b79e7, data=None),
 Node(tag=Comedy Club, identifier=4bf58dd8d48988d18e941735, data=None),
 Node(tag=Concert Hall, identifier=5032792091d4c4b30a586d5c, data=None),
 Node(tag=Country Dance Club, identifier=52e81612bcbc57f1066b79ef, data=None),
 Node(tag=Disc Golf, identifier=52e81612bcbc57f1066b79e8, data=None),
 Node(tag=Exhibit, identifier=56aa371be4b08b9a8d573532, data=None),
 Node(tag=General Entertainment, identifier=4bf58dd8d48988d1f1931735, data=None),
 Node(tag=Go Kart Track, identifier=52e81612bcbc57f1066b79ea, data=

In [25]:
def get_SIM_WP(tree_loc, data): 
    sim_wp=[]  #[[t1,[t2,value],[t3,value]...]...]
    for i in tqdm(range(len(data))):
        sim_wp_col=[]
        for j in range(len(data)):
            if i==j:
                continue
            else:
                parent_id=findparent(data.loc[i,"id"],data.loc[j,"id"],tree_loc)
                n1=findpath(data.loc[i,"id"],parent_id,tree_loc)
                n2=findpath(data.loc[j,"id"],parent_id,tree_loc)
                n3=tree_loc.depth(parent_id)
                sim=2*n3/(n1+n2+2*n3)
                n=[data.loc[j,"id"],sim]
            sim_wp_col.append(n)
        sim_wp_col=sorted(sim_wp_col,key=(lambda x:x[1]),reverse=True)   #对数值进行排序
        sim_wp_col1=[item[0]+"@"+str(item[1]) for item in sim_wp_col]     #按格式存储
        sim_wp_col=[data.loc[i,"id"]]+sim_wp_col1
        sim_wp.append(sim_wp_col)

    df = pd.DataFrame(sim_wp)
    return df

In [26]:
def get_SIM_LC(tree_loc, data):
    sim_lc=[]  #[[t1,[t2,value],[t3,value]...]...]为数据格式
    for i in tqdm(range(len(data))):
        sim_lc_col=[]
        for j in range(len(data)):
            if i==j:
                continue
            else:
                n=findpath(data.loc[i,"id"],data.loc[j,"id"],tree_loc)
                sim=-math.log(n/10)
                n=[data.loc[j,"id"],sim]
                sim_lc_col.append(n)
        sim_lc_col=sorted(sim_lc_col,key=(lambda x:x[1]),reverse=True)   #对数值进行排序
        sim_lc_col1=[item[0]+"@"+str(item[1]) for item in sim_lc_col]
        sim_lc_col=[data.loc[i,"id"]]+sim_lc_col1
        sim_lc.append(sim_lc_col)
    df = pd.DataFrame(sim_lc)
    return df

In [27]:
def get_SIM_Lin(tree_loc, data, ic_seco_dict):
    sim_lin=[]
    for i in tqdm(range(len(data))):
        sim_lin_col=[]
        for j in range(len(data)):
            if i==j:
                continue
            id1=data.loc[i,"id"]
            id2=data.loc[j,"id"]
            parent=findparent(id1,id2,tree_loc)     #两id的最小共同父节点
            ic_parent=ic_seco_dict[parent]
            ic_t1=ic_seco_dict[id1]
            ic_t2=ic_seco_dict[id2]
            sim=2*ic_parent/(ic_t1+ic_t2)
            sim_lin_col.append([id2,sim])
        sim_lin_col=sorted(sim_lin_col,key=(lambda x:x[1]),reverse=True)
        sim_lin_col1=[item[0]+"@"+str(item[1]) for item in sim_lin_col]
        sim_lin_col=[data.loc[i,"id"]]+sim_lin_col1
        sim_lin.append(sim_lin_col)
    df = pd.DataFrame(sim_lin)
    return df

In [28]:
def get_SIM_JC(tree_loc, data, ic_sanchez_dict):
    sim_jc=[]
    for i in tqdm(range(len(data))):
        sim_jc_col=[]
        for j in range(len(data)):
            if i==j:
                continue
            id1=data.loc[i,"id"]
            id2=data.loc[j,"id"]
            parent=findparent(id1,id2,tree_loc)
            ic_parent=ic_sanchez_dict[parent]
            ic_t1=ic_sanchez_dict[id1]
            ic_t2=ic_sanchez_dict[id2]
            sim=ic_t1+ic_t2-(2*ic_parent)
            sim=1/sim
            sim_jc_col.append([data.loc[j,"id"],sim])
        sim_jc_col=sorted(sim_jc_col,key=(lambda x:x[1]),reverse=True)
        sim_jc_col1=[item[0]+"@"+str(item[1]) for item in sim_jc_col]
        sim_jc_col=[data.loc[i,"id"]]+sim_jc_col1
        sim_jc.append(sim_jc_col)
    df = pd.DataFrame(sim_jc)
    return df

In [29]:
def get_IC_Seco(tree, data):
    ic_seco_dict_name=[]       #IC_seco 的字典！！！
    ic_seco_dict_values=[]
    for i in range(len(data)):
        id=data.loc[i,"id"]
        ic_seco_dict_name=ic_seco_dict_name+[id]
        hypo=len(tree_loc.subtree(id))-1   #id下所有结点
        max_types=938   #len(tree_loc)    #所有节点
        ic=1-math.log(abs(hypo)+1)/math.log(max_types)
        ic_seco_dict_values=ic_seco_dict_values+[ic]
    ic_seco_dict=dict(zip(ic_seco_dict_name,ic_seco_dict_values))
    ic_seco_dict.update({'root':1-math.log(abs(max_types))/math.log(max_types)})   #data文件中没有root，单独把root加入
    return ic_seco_dict

In [30]:
def get_IC_Sanchez(tree, data):
    ic_sanchez_dict_name=[]
    ic_sanchez_dict_values=[]
    for i in range(len(data)):
        id=data.loc[i,"id"]
        ic_sanchez_dict_name=ic_sanchez_dict_name+[id]
        leaves=len(tree_loc.leaves(id))   #id下所有叶子结点
        subsumers=0
        if id=="root":
            subsumers=1
        else:
            subsumers=tree_loc.depth(id)   #比id更概括的类型
        max_leaves=865            #len(tree_loc.leaves('root'))   所有叶子数
        ic=-math.log((abs(leaves)/abs(subsumers)+1)/(max_leaves+1))
        ic_sanchez_dict_values=ic_sanchez_dict_values+[ic]
    ic_sanchez_dict=dict(zip(ic_sanchez_dict_name,ic_sanchez_dict_values))
    ic_sanchez_dict.update({'root':-math.log((abs(max_leaves)+1)/(max_leaves+1))})  #data文件中没有root，单独把root加入
    return ic_sanchez_dict

In [31]:
data = pd.read_excel("./data/foursquare_category.xlsx")
IC_Sanchez_dict = get_IC_Sanchez(tree_loc, data)
IC_Seco_dict = get_IC_Seco(tree_loc, data)

In [32]:
SIM_WP = get_SIM_WP(tree_loc, data)
SIM_LC = get_SIM_LC(tree_loc, data)
SIM_Lin_Sanchez = get_SIM_Lin(tree_loc, data, IC_Sanchez_dict)
SIM_Lin_Seco = get_SIM_Lin(tree_loc, data, IC_Seco_dict)
SIM_JC_Sanchez = get_SIM_JC(tree_loc, data, IC_Sanchez_dict)
SIM_JC_Seco = get_SIM_JC(tree_loc, data, IC_Seco_dict)

100%|████████████████████████████████████████████████████████████████████████████████| 937/937 [00:36<00:00, 25.46it/s]


### 下面不是

In [33]:
def output_695_correct_responses(data, name):
    queries = list(data[0])
    correct_responses = list(data[1])
    for i in range(len(correct_responses)):
        correct_responses[i] = correct_responses[i].split('@')[0]
    qr_dict = dict(zip(queries, correct_responses))
#     print(qr_dict)

    with open('./data/tsmc2014_nyc_categories_used.txt', 'r', encoding='utf-8') as f:
        categories = f.readlines()
    for i in range(len(categories)):
        categories[i] = categories[i].strip()
    
    csv_data = pd.read_excel("./data/foursquare_category.xlsx")
    responses = []
    for category in categories:
        query = csv_data.loc[lambda x:x['name'] == category, 'id'].values[0]
        response_id = qr_dict[query]
        response = csv_data.loc[lambda x:x['id'] == response_id, 'name'].values[0]
        responses.append(response)

    with open('./data/foursquare_tsmc2014_nyc_ground_truth/' + name + '.txt', 'a') as f:
        for r in responses:
            f.write(r + "\n")

In [34]:
output_695_correct_responses(SIM_WP, 'SIM_WP')
# output_695_correct_responses(SIM_LC, 'SIM_LC')
# output_695_correct_responses(SIM_Lin_Sanchez, 'SIM_Lin_Sanchez')
# output_695_correct_responses(SIM_Lin_Seco, 'SIM_Lin_Seco')
# output_695_correct_responses(SIM_JC_Sanchez, 'SIM_JC_Sanchez')
# output_695_correct_responses(SIM_JC_Seco, 'SIM_JC_Seco')